This Code calcuates the viscous evolution of a circum-planetary particle disk with radially varying viscosity. The results of the code are used for the results in Fig 3b.

Please note that this code was used in the paper to estimate the time-scale over which the disk spreads out to the top of the Martian atmosphere, not for any subsequent evolution. 

It was designed for simplicity (not speed) and there are choices related to boundary conditions at the edge of the radial grid that should be carefully thought about for other applications.


In [1]:
# Define the variables in this section ...

% pylab
from astropy import constants
from scipy import integrate

G_val = constants.G.si.value

M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3    # Mars, m
rho_mars = 3933         #kg/m^3
V_phobos = 5689*1e9     # +/- 60*1e9 m^3
M_phobos = 1.065*(10**16.) # Phobos, +/- .015 kg
R_phobos = 11.3*1e3  # Phobos, m
rho_phobos = 1860. # +/- 31 kg/m^3
ecc_phobos =  0.01511
a_phobos = 9378.*1e3 # Semi-major Axis, m 
w_phobos = 1./27562. # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# base parameters of the ring - 
part_size   = 1.                      # in m, mean particle size
a_r_initial = 1.1                     # Initial centre location of the disk (in R_Mars)
Ring_width_int_fac  = .02*a_r_initial # initial half width = 0.02*a_r_initial (in R_Mars)

num_r_points = 200                       # Number of radial grid points
r_grid = linspace(.9,2.,num_r_points)    # grid of 'r' from .9 to 2. R_mars 
#           (this should be modified depending on the ring's initial location to allow proper evolution of the disk)

inner_bndry_cond = .9                    # Inner boundary location, these are used later in the code ..
outer_bndry_cond = 2.                    # Outer boundary location

Disk_mass_frac_Phobos = 1.               # Fraction of Phobos as disk mass

# Note that we define the initial density profile of the disk to be a Gaussian 
# and the total disk mass is equal to Phobos Mass (or change it using Disk_mass_frac_Phobos) 

In [3]:
# Define function to calculate the viscosity of the disk at a given location (using the eqns in the paper)

y_local     =  .35
y_nonlocal  = 1.

def func_visc(a_r_part,part_size,Surf_dens) :
    sigma_part = pi*part_size**2.
    M_part = (4.*pi/3.)*(part_size**3.)*(3000.) # mass of each particle 
    N_colmn_dens = Surf_dens/M_part
    Opt_depth = N_colmn_dens*sigma_part                 # tau
    
    ## Version with values varying across the ring ....
    n_mean = sqrt(G_val*M_mars/(a_r_part*R_mars)**3.)
    r_hill = a_r_part*R_mars*(2.*M_part/3./M_mars)**(1./3.)
    r_h_star = r_hill/2./part_size
    
    vel_disp1 = 2.*part_size*n_mean
    vel_disp2 = sqrt(2.*G_val*M_part/part_size)
    
    vel_disp_final = max(vel_disp1,vel_disp2)
    if Surf_dens < 0.001 :
        return 0.
    Q_value = (n_mean/G_val/3.36)*vel_disp_final/Surf_dens
    v_trans = y_local*(vel_disp_final**2.)*Opt_depth/(1. + Opt_depth**2.)/n_mean
    v_gravity = 0.
    v_nonlocal = y_nonlocal*(part_size**2.)*Opt_depth*n_mean
    v_total = v_trans + v_nonlocal + v_gravity
    #print(Q_value)
    if Q_value < 2. :
        vel_disp_final = Surf_dens*3.36*G_val/n_mean
        Q_value = 2.
        C_wake_term = min(53.*(r_h_star**5.),30.)
        v_trans    = .5*C_wake_term*(G_val*Surf_dens)**2./n_mean**3.
        v_gravity  = .5*C_wake_term*(G_val*Surf_dens)**2./n_mean**3.
        v_nonlocal = y_nonlocal*(part_size**2.)*Opt_depth*n_mean
        v_total  = v_trans + v_nonlocal  + v_gravity 

    return v_total

###################################################################
################# Section on initial conditions ... 
dr_val = r_grid[1] - r_grid[0]

def gaussian_func(x, mu, sig): # mu - mean, sig - sigma ... (Normalized so total Mass is 1.)
    norm = sqrt(2.*pi)*sig
    return exp(-(x - mu)**2./2./sig**2.)/norm

def gaussian_funcD(x, mu, sig): # mu - mean, sig - sigma
    norm = sqrt(2.*pi)*sig
    return 2.*pi*x*exp(-(x - mu)**2./2./sig**2.)/norm # Integrand function for the total mass

mass_total,err = integrate.quad(gaussian_funcD,min(r_grid),max(r_grid),args=(a_r_initial,Ring_width_int_fac),epsabs=1e-15)

# Note that we have fixed the initial disk mass to be equal to Phobos Mass
scale_fact_dens = M_phobos*Disk_mass_frac_Phobos/mass_total/R_mars/R_mars               

# unit is kg/m^2, Surface mass density
dens_init = scale_fact_dens*gaussian_func(r_grid,a_r_initial , Ring_width_int_fac) 
dens_init[dens_init < 0.001] = 0. # Set a floor for the surface mass density


In [4]:
# Set up functions for the viscosity calculations as well as computing the mass fluxes into and out of each bin
from scipy import interpolate

###### Functions for density evolution ...

def visc_func(r_grid,part_size,dens_init,num_r_points):
    init_viscosity = r_grid.copy()*0.0
    for kp in range(0,num_r_points):
        init_viscosity[kp] = func_visc(r_grid[kp],part_size,dens_init[kp])
    return init_viscosity

def compute_flux(init_viscosity,dens_init,r_grid,dr_val):
    ## Next Step, Calculate the time evolution
    Flux_term_integrand = init_viscosity*dens_init*sqrt(r_grid*R_mars) 
    Flux_term_grad = gradient(Flux_term_integrand,dr_val*R_mars)  
    # Calculates - d/dr (v*sigma*sqrt(r)) - Goes left to right
    Flux_term_total = Flux_term_grad*sqrt(r_grid*R_mars)*6.*pi
    
    # Note that in detail, we want a staggered grid for the fluxes into/out of the bin -
    # So lets define a local interpolation function for flux(r)    
    f_flux_r_term = interpolate.interp1d(r_grid, Flux_term_total,fill_value=0.,bounds_error=False) 
    # Note, be careful to use it only for something within the range of r_grid (set outside regions to be 0.)
    # effectuvely : flux_dsigma[r_grid < inner_bndry_cond] =  0. # No flux at the bndry (atmosphere ~ 1 Mars radii, allow a few inner bins to prevent strong bndry effect on evolution)
    # effectuvely : flux_dsigma[r_grid > outer_bndry_cond] = 0. # No flux at the bndry (set an effective outer bndry to restrict computational domain, the choice marginally affect results at best)
    new_r_grid1 = r_grid + dr_val/2.
    new_r_grid2 = r_grid - dr_val/2.
    flux_dsigma = f_flux_r_term(new_r_grid1) - f_flux_r_term(new_r_grid2)
    Bin_area_func = 2.*pi*R_mars*r_grid*dr_val*R_mars
    return flux_dsigma/Bin_area_func


In [8]:
# Integration section of the code - Specify the time_steps to do, max time step

import time
steps_to_do = 5000              # specify the time-steps to take  
dt_init = 8e2*3.154e7           # unit is seconds , this is the maximum value of the time-step 

dens_all_time = zeros([num_r_points,steps_to_do/10+1]) # variable to save the density
t_now = zeros([steps_to_do/10+1,1])
t_now[0] = dt_init

dens_temp_var = dens_init.copy()
t_now_temp = 0.
dens_all_time[:,0] = dens_init.copy()
mass_all_times = zeros([steps_to_do,1])
t_all_times = zeros([steps_to_do,1])
accreted_mass = zeros([steps_to_do/10+1])
accreted_massV2 = zeros([steps_to_do/10+1])

accreted_mass_tmp = 0.
accreted_mass[0] = accreted_mass_tmp


dt = 8e4*3.154e7           # unit is seconds, this is the time-step at t=0

for i in range(0,steps_to_do-1):
        #t1 = time.time()
        visc_all = visc_func(r_grid,part_size,dens_temp_var,num_r_points)
        visc_all2 = visc_all.copy()
        visc_all2[visc_all2<=0] = 1e-8 
        max_dt_allowed = min(((dr_val*R_mars)**2.)/visc_all2)/dt
        dt = min(.5*max_dt_allowed*dt,dt_init)  # A simple viscous evolution estimate to regulate the time-steps
        # Note that dt can adaptively change during the simulation ...
        
        flux_term_r = compute_flux(visc_all,dens_temp_var,r_grid,dr_val)
        dens_temp_var += flux_term_r*dt
        mass_all_times[i] = sum(dens_temp_var*2.*pi*dr_val*R_mars*r_grid*R_mars)
        accreted_mass_tmp += sum(dens_temp_var[r_grid<=inner_bndry_cond]*2.*pi*dr_val*R_mars*r_grid[r_grid<=inner_bndry_cond]*R_mars)
        
        dens_temp_var[r_grid<=inner_bndry_cond] = 0.
        t_now_temp +=dt
        t_all_times[i] = t_now_temp 
        if (steps_to_do%10.) == 0. :  # Save the density grid every 10 time-steps
            dens_all_time[:,i/10+1] = dens_temp_var
            accreted_mass[i/10+1] = accreted_mass_tmp
            t_now[i/10+1] = t_now_temp  # in Seconds 
        
        
print("Final_Time(Myr) :",(t_now_temp/3.154e7/1e6))

# Save the density grid for subsequent plotting (See Code-Disk_evolution-Plot_3b.ipynb)
savez('output_1p1_run',t_now/3.154e7/1e6,dens_all_time,r_grid) 




Final_Time(Myr) : 3.9992


Please again note that users should carefully go through the code and boundary condition choices before using it for any applications. 